<img src="../gfx/euro2.png" alt="Euro" style="display: block; margin: auto; width: 100%;" />

**ONCFM - Organisation nationale de lutte contre le faux-monnayage** est une organisation publique ayant pour objectif de mettre en place des méthodes d'identification des contrefaçons des billets en euros.
Dans le cadre de cette lutte, nous souhaitons mettre en place un algorithme qui soit capable de différencier automatiquement les vrais des faux billets.

<div style="text-align:center;"><h1>Application de détection de faux billets</h1></div>

<img src="../gfx/sep.jpg" alt="Barre">

## Chargement

***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import io
from IPython.display import display

sns.color_palette("viridis")
sns.set_theme(style="darkgrid")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import _base

In [2]:
# Chargement du modèle
le_modele_path = 'le_modele_1.0.pkl'

<img src="../gfx/sep.jpg" alt="Barre">

## 1 - Chargement du fichier de données

In [3]:
data = pd.read_csv("../data/billets_production.csv")
data.head()

,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5


## 2 - Mise en conformité des données

> Traitement de la conformité des variables.

> Traitement des données manquantes par régression linéaire.

In [4]:
class ColumnConformityTransformer:
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Vérifier si les colonnes sont présentes dans le DataFrame
        missing_columns = set(self.columns).difference(X.columns)
        if not missing_columns:
            return X
        else:
            raise ValueError(f"Colonnes manquantes : {missing_columns}")
        
class MissingValuesRegressor(_base.TransformerMixin):
    def __init__(self, columns=None):
        super().__init__()
        self.columns = columns

    def fit(self, X, y=None):
        # Si des colonnes sont spécifiées, on les filtre
        if self.columns is not None:
            X = X[self.columns]

        # Entraînement d'un modèle de régression linéaire pour chaque colonne dave des valeurs manquantes
        self.models = {}
        for col in X.columns:
            mask = X[col].isna()
            if mask.any():
                model = LinearRegression()
                X_train = X[~mask]
                y_train = X_train[col]
                model.fit(X_train.drop(col, axis=1), y_train)
                self.models[col] = model
        return self

    def transform(self, X):
        # Si des colonnes sont spécifiées, on les filtre
        if self.columns is not None:
            X = X[self.columns]

        # Prédiction des valeurs manquantes pour chaque colonne
        X_transformed = X.copy()
        for col in X_transformed.columns:
            mask = X_transformed[col].isna()
            if mask.any() and col in self.models:
                # Prédire les valeures manquantes
                X_filled = self.models[col].predict(X_transformed[~mask].drop(col, axis=1))
                X_transformed.loc[mask, col] = X_filled
        return X_transformed
    
class TrainedModelApplier(_base.TransformerMixin):
    def __init__(self, model_path):
        self.model_path = model_path

    def fit(self, X, y=None):
        # Chargement du modèle déjà entraîné
        self.model = joblib.load(self.model_path)
        return self
        
    def transform(self, X):
        # Appliquer le modèle pour détecter les faux billets
        predictions = self.model.predict(X)
        return pd.DataFrame({'prediction': predictions})

In [5]:
# Définition des colonnes attendues
colonnes = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']

In [6]:
# Créer le pipeline
pipeline = Pipeline([
    ('column_checker', ColumnConformityTransformer(columns=colonnes)),
    ('missing_imputer', MissingValuesRegressor(columns=colonnes)),
    ('scaler', StandardScaler()),
    ('trained_model', TrainedModelApplier(le_modele_path))
])

pred = pipeline.fit_transform(data)
pred

,prediction
0,0
1,0
2,0
3,1
4,1
